<a href="https://colab.research.google.com/github/graphtrek/stockforecast/blob/main/graphtrek_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [473]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MAE
from tensorflow.keras.layers import Dense, Dropout
from tensorflow import keras
from datetime import datetime, timedelta
import pandas as pd
import os, time
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MAE
from tensorflow.keras.layers import Dense, Dropout
from tensorflow import keras
from datetime import datetime, timedelta
import pandas as pd
import os, time
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from keras.preprocessing.sequence import TimeseriesGenerator
from pathlib import Path

In [474]:
ticker = "VTI" # QQQ, SPY, VTI JETS, XLE, XLF, XLNX, ARKK, ARKG, ARKQ, ARKF, PYPL, V
look_back = 3 #  number of past days we want to use to predict 1 day in the future.
max_data_size = 730 # ~2 years
split_percent = 0.90 # use 90% of the data  for train
INFO = 1
DEBUG = 2
TRACE = 3
print_level = INFO 
indicators_model_file_path = '/content/drive/MyDrive/models/' + ticker + '_' + str(look_back) + '_indicators_08.h5'
nr_of_features = 3
look_forward = 1
epochs = 420
learning_rate=0.0042
num_prediction = look_back * 4
apikey = '3F4URDEKOPLFH25T'
np.random.seed(42)

In [475]:
rsi_url = 'https://www.alphavantage.co/query?function=RSI&datatype=json&symbol='+ticker+'&interval=daily&time_period='+str(look_back)+'&series_type=close&apikey=' + apikey
print(rsi_url)
rsi_api_response = pd.read_json(rsi_url)
#print(rsi_api_response)
rsi_api_data = rsi_api_response['Technical Analysis: RSI']
#print(rsi_api_data)
rsi_api_data = rsi_api_data.drop(
    index=['1: Symbol','2: Indicator','3: Last Refreshed','4: Interval','5: Time Period','6: Series Type','7: Time Zone']);
#print(rsi_api_data)

rsi_data = []
for key, value in rsi_api_data.items():
  date = key
  rsi = float(value.get('RSI'))
  rsi_data.append([date,rsi])

last_rsi_data =  str(rsi_data[0])
first_rsi_data =  str(rsi_data[-1])
print('rsi_data_length:', len(rsi_data), 'first_rsi_data:', first_rsi_data, 'last_rsi_data:', last_rsi_data)  

https://www.alphavantage.co/query?function=RSI&datatype=json&symbol=VTI&interval=daily&time_period=3&series_type=close&apikey=3F4URDEKOPLFH25T
rsi_data_length: 5150 first_rsi_data: ['2001-06-05', 100.0] last_rsi_data: ['2021-11-19', 40.4515]


In [476]:
stoch_url = 'https://www.alphavantage.co/query?function=STOCH&symbol='+ticker+'&interval=daily&apikey=' + apikey
print(stoch_url)
stoch_api_response = pd.read_json(stoch_url)
stoch_api_data = stoch_api_response['Technical Analysis: STOCH']
#print(rsi_api_data)
stoch_api_data = stoch_api_data.drop(
    index=['1: Symbol','2: Indicator','3: Last Refreshed','4: Interval','5.1: FastK Period','5.2: SlowK Period','5.3: SlowK MA Type','5.4: SlowD Period','5.5: SlowD MA Type','6: Time Zone'])
#print(stoch_api_data)
stoch_data = []
for key, value in stoch_api_data.items():
  date = key
  SlowD = float(value.get('SlowD'))
  SlowK = float(value.get('SlowK'))
  stoch_data.append([date,SlowD,SlowK])

last_stoch_data =  str(stoch_data[0])
first_stoch_data =  str(stoch_data[-1])
print('stoch_data_length:', len(stoch_data), 'first_stoch_data:', first_stoch_data, 'last_stoch_data:', last_stoch_data)  

https://www.alphavantage.co/query?function=STOCH&symbol=VTI&interval=daily&apikey=3F4URDEKOPLFH25T
stoch_data_length: 5145 first_stoch_data: ['2001-06-12', 48.2257, 33.8565] last_stoch_data: ['2021-11-19', 63.0967, 50.4436]


In [477]:
stock_url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol='+ticker+'&outputsize=full&apikey=' + apikey
print(stock_url)
stock_api_response = pd.read_json(stock_url)
stock_api_data = stock_api_response['Time Series (Daily)']
#print(stock_api_data)
stock_api_data = stock_api_data.drop(index=['1. Information','2. Symbol','3. Last Refreshed','4. Output Size','5. Time Zone']);

stock_data = []
split_coefficient = 1

for key, value in stock_api_data.items():
  date = key
  open = float(value.get('1. open'))
  high = float(value.get('2. high'))
  low = float(value.get('3. low'))
  close = float(value.get('4. close'))
  adjusted_close = float(value.get('5. adjusted close'))
  volume = int(value.get('6. volume'))
  divident = float(value.get('7. dividend amount'))

  if float(value.get('8. split coefficient')) > 1:
    split_coefficient = float(value.get('8. split coefficient'))
    
  open = open / split_coefficient
  high = high /split_coefficient
  close = close / split_coefficient
  low = low / split_coefficient
  
  stock_data.append([
      date,
      close,
      volume,
      high,
      low,
      open,
      divident
      ])

last_date =  str(stock_data[0][0])
print('data length:', len(stock_data), 'last_date:', last_date)

last_stock_data =  str(stock_data[0])
first_stock_data =  str(stock_data[-1])
print('stock_data_length:', len(stock_data), 'first_stock_data:', first_stock_data, 'last_stock_data:', last_stock_data)

https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=VTI&outputsize=full&apikey=3F4URDEKOPLFH25T
data length: 5153 last_date: 2021-11-19
stock_data_length: 5153 first_stock_data: ['2001-05-31', 57.4, 2457200, 57.75, 57.25, 57.25, 0.0] last_stock_data: ['2021-11-19', 241.48, 3354681, 242.62, 241.3302, 242.09, 0.0]


In [478]:
data_size = np.min([len(rsi_data), len(stoch_data), len(stock_data)])
if max_data_size < data_size:
  rsi_data = np.flip(rsi_data[:max_data_size],axis=0)
  stoch_data = np.flip(stoch_data[:max_data_size],axis=0)
  stock_data = np.flip(stock_data[:max_data_size],axis=0)
else:
  rsi_data = np.flip(rsi_data[:data_size],axis=0)
  stoch_data = np.flip(stoch_data[:data_size],axis=0)
  stock_data = np.flip(stock_data[:data_size],axis=0)

dates = stock_data[:,0]

rsi_array = rsi_data[:,1]
stoch_slowD_array = stoch_data[:,1]
stoch_slowK_array = stoch_data[:,2]
#np.expand_dims(dates, axis=1)[:10]

indicators_lstm_data = np.expand_dims(rsi_array, axis=1).astype(float)
indicators_lstm_data = np.append(indicators_lstm_data, np.expand_dims(stoch_slowD_array, axis=1),axis=1).astype(float)
indicators_lstm_data = np.append(indicators_lstm_data, np.expand_dims(stoch_slowK_array, axis=1),axis=1).astype(float)
#lstm_data = np.reshape(lstm_data, (-1, 2))
indicators_lstm_data[:10]

array([[ 69.654600000000,  68.153500000000,  86.354800000000],
       [ 70.554800000000,  82.284600000000,  88.318500000000],
       [ 30.450100000000,  83.849400000000,  76.875000000000],
       [ 68.236900000000,  80.444800000000,  76.140900000000],
       [ 74.721500000000,  75.662300000000,  73.970900000000],
       [ 80.786300000000,  80.545200000000,  91.523900000000],
       [ 83.529000000000,  86.061300000000,  92.689000000000],
       [ 86.548500000000,  93.607900000000,  96.610800000000],
       [ 86.717100000000,  95.462200000000,  97.086900000000],
       [ 56.812600000000,  94.145800000000,  88.739600000000]])

In [479]:
stock_df = pd.DataFrame(stock_data,columns=['Date','Close','Volume','High','Low','Open','Divident'])
stock_df['50MA'] = stock_df['Close'].rolling(50).mean()
stock_df['100MA'] = stock_df['Close'].rolling(100).mean()
stock_df['200MA'] = stock_df['Close'].rolling(200).mean()

In [480]:
indicators_df = pd.DataFrame(rsi_data,columns=['Date','RSI'])

split = int(split_percent*len(rsi_data))
if len(rsi_data) - split < look_back:
  split = look_back
split


indicators_df_train = indicators_df.iloc[:split]
indicators_df_test = indicators_df.iloc[split:]

indicators_lstm_train_data = indicators_lstm_data[:split]
indicators_lstm_test_data = indicators_lstm_data[split:]

In [481]:
np.set_printoptions(formatter={'float': '{: 0.12f}'.format})

indicators_scaler = MinMaxScaler(feature_range=(0, 1))
indicators_scaler = indicators_scaler.fit(indicators_lstm_data)

indicators_lstm_train_data_scaled = indicators_scaler.transform(indicators_lstm_train_data)
if print_level >= TRACE:
  print(indicators_lstm_train_data_scaled[:look_back+1])
  print(indicators_lstm_train_data_scaled.shape)

indicators_lstm_test_data_scaled = indicators_scaler.transform(indicators_lstm_test_data)
if print_level >= TRACE:
  print(indicators_lstm_test_data_scaled[:look_back+1])
  print(indicators_lstm_test_data_scaled.shape)

In [482]:
indicators_train_generator = TimeseriesGenerator(indicators_lstm_train_data_scaled, indicators_lstm_train_data_scaled, sampling_rate=1, length=look_back, batch_size=64)

if print_level >= TRACE:
  print('Samples: %d' % len(indicators_train_generator))
  # print each sample
  for i in range(2):
    x, y = indicators_train_generator[i]
    print('%s => %s' % (x, y))

In [483]:
indicators_model = None
try:
  indicators_model = keras.models.load_model(indicators_model_file_path)
  modified = os.path.getmtime(indicators_model_file_path)

  print(pd.to_datetime(last_date).date() - datetime.fromtimestamp(modified).date() )
  print('Loaded', indicators_model_file_path , ' model train date:',datetime.fromtimestamp(modified).date() , 'last date:', last_date)
except:
  indicators_model = None
  print('Model ' + indicators_model_file_path + ' does not exists.')

Model /content/drive/MyDrive/models/VTI_3_indicators_08.h5 does not exists.


In [484]:
if indicators_model is None:
  indicators_model = Sequential()
  indicators_model.add(LSTM(units=50, activation='tanh', input_shape=(look_back, nr_of_features), return_sequences=True))
  indicators_model.add(Dropout(0.2))
  indicators_model.add(LSTM(units=50, return_sequences=True, activation="tanh"))
  indicators_model.add(Dropout(0.2))
  indicators_model.add(LSTM(units=50, return_sequences=True, activation="tanh"))
  indicators_model.add(Dropout(0.2))
  indicators_model.add(LSTM(units=50, activation="tanh"))
  indicators_model.add(Dropout(0.2))
  indicators_model.add(Dense(nr_of_features))

  indicators_model.compile(
      loss=MAE,
      optimizer=Adam(learning_rate=learning_rate),
      metrics=["mae"]
  )
    
  #model.summary()
  print('Model ' + indicators_model_file_path + ' compiled.')

  indicators_modelo = indicators_model.fit(indicators_train_generator, epochs=epochs, verbose=0 )
  
  indicators_model.save(indicators_model_file_path)
  print('Saved model ' + indicators_model_file_path)

  if print_level >= DEBUG:
    plt.plot(indicators_modelo.history['loss'])
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()

Model /content/drive/MyDrive/models/VTI_3_indicators_08.h5 compiled.
Saved model /content/drive/MyDrive/models/VTI_3_indicators_08.h5


In [485]:
indicators_test_generator = TimeseriesGenerator(indicators_lstm_test_data_scaled, indicators_lstm_test_data_scaled,sampling_rate=1, length=look_back, batch_size=look_back)

print('Samples: %d' % len(indicators_test_generator))
# print each sample
if print_level >= TRACE:
  for i in range(2):
    x, y = indicators_test_generator[i]
    print('%s => %s' % (x, y))

Samples: 24


In [486]:
#Make prediction
indicators_prediction_scaled = indicators_model.predict(indicators_test_generator)

if print_level >= DEBUG:
  np.set_printoptions(formatter={'float': '{: 0.12f}'.format})
  print(indicators_prediction_scaled[:5])

indicators_prediction = indicators_scaler.inverse_transform(indicators_prediction_scaled)
if nr_of_features == 1:
   rsi_prediction = indicators_prediction.reshape((-1))
else:
  rsi_prediction = indicators_prediction[:,0].reshape((-1))


if print_level >= DEBUG:
  np.set_printoptions(formatter={'float': '{: 0.2f}'.format})
  print('indicators_prediction shape:',indicators_prediction.shape)

  for x in range(look_back + 1):
    print('pred:',indicators_prediction[x], 'test:', indicators_lstm_test_data[x])


In [487]:
def predict(num_prediction, model):
    prediction_list = indicators_lstm_test_data_scaled[-look_back:]
    #print('prediction_list',prediction_list)
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        #print(x)
        x = x.reshape((1, look_back, nr_of_features))
        out = model.predict(x)
        #print(x,out) 
        prediction_list = np.append(prediction_list, out, axis=0)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(last_date,num_prediction):
    us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1,freq=us_bd).tolist()
    return prediction_dates

indicators_forecast_scaled = predict(num_prediction, indicators_model)
#print(forecast_scaled)
indicators_forecast_dates = predict_dates(indicators_df['Date'].values[-1],num_prediction)

indicators_forecast = indicators_scaler.inverse_transform(indicators_forecast_scaled.reshape((-1,nr_of_features)))
if nr_of_features == 1:
   rsi_forecast = indicators_forecast.reshape((-1))
else:
  rsi_forecast = indicators_forecast[:,0].reshape((-1))
print('indicators_forecast:',indicators_forecast)
print('rsi_forecast:',rsi_forecast)

indicators_forecast: [[ 40.451500000000  63.096700000000  50.443600000000]
 [ 60.444577142334  57.537342219883  49.221236813432]
 [ 67.321040043449  53.292475414038  53.290249951959]
 [ 81.378617595291  59.816680131698  74.653824139458]
 [ 82.657973167038  71.063814850974  85.799126139700]
 [ 84.901477789497  82.618822423232  91.226002309835]
 [ 84.876193248177  87.782561069101  89.751331060481]
 [ 84.320373121262  87.994885457683  87.458233849800]
 [ 83.766329714203  85.729754508787  84.363789625788]
 [ 82.944124747849  82.510815204746  80.789626872259]
 [ 82.102440364456  79.102095660836  76.766124854630]
 [ 81.157401313782  75.154435421169  72.050902332228]
 [ 80.034702005005  70.589253605175  66.805533317482]]
rsi_forecast: [ 40.451500000000  60.444577142334  67.321040043449  81.378617595291
  82.657973167038  84.901477789497  84.876193248177  84.320373121262
  83.766329714203  82.944124747849  82.102440364456  81.157401313782
  80.034702005005]


In [488]:
chart_days = len(indicators_df_test) * 2
trace1 = go.Candlestick(
    x = stock_df.tail(chart_days)['Date'],
    open = stock_df.tail(chart_days)['Open'],
    high = stock_df.tail(chart_days)['High'],
    low = stock_df.tail(chart_days)['Low'],
    close = stock_df.tail(chart_days)['Close'],
    name = ticker + ' Price'
)

trace2 = go.Scatter(
    x = stock_df.tail(chart_days)['Date'],
    y = stock_df.tail(chart_days)['50MA'],
    mode='lines',
    name ='50MA'
)

trace3 = go.Scatter(
    x = stock_df.tail(chart_days)['Date'],
    y = stock_df.tail(chart_days)['100MA'],
    mode='lines',
    name ='100MA'
)

trace4 = go.Scatter(
    x = stock_df.tail(chart_days)['Date'],
    y = stock_df.tail(chart_days)['200MA'],
    mode='lines',
    name ='200MA'
)


#'Date','Close','Volume','High','Low','Open','Divident'
open_price =  str(stock_data[-1][5])
low_price =  str(stock_data[-1][3])
high_price =  str(stock_data[-1][4])
close_price =  str(stock_data[-1][1])
rsi_forecast_mean = np.round(np.mean(rsi_forecast))

go.Layout(
    title = ticker + ' Date:' + last_date + ' Open:' + open_price + ' High:' + high_price + ' Low:' + low_price + ' Close:' + close_price,
    xaxis = {'title' : "Dates"},
    yaxis = {'title' : "Close Price ($)"},
    height = 450
)

trace_train = go.Scatter(
  x = indicators_df_train.tail(len(indicators_df_test))['Date'],
  y = indicators_df_train.tail(len(indicators_df_test))['RSI'],
  mode='lines',
  name ='RSI'
)

trace_test = go.Scatter(
  x = indicators_df_test['Date'],
  y = indicators_df_test['RSI'],
  mode='markers+lines',
  name ='Test'
)

trace_prediction = go.Scatter(
    x = indicators_df_test['Date'],
    y = rsi_prediction,
    mode='markers+lines',
    name ='Prediction'
)

trace_forecast = go.Scatter(
    x = indicators_forecast_dates,
    y = rsi_forecast,
    mode='markers+lines',
    name ='Forecast'
)


tradingview_link = '<a href="https://in.tradingview.com/chart/66XmQfYy/?symbol=' + ticker +'">' + ticker +'</a>'
stock_title = tradingview_link + ' Date:' + last_date + ' Open:' + open_price + ' High:' + high_price + ' Low:' + low_price + ' Close:' + close_price
indicators_title = tradingview_link + ' RSI('+ str(look_back)+ ') Date:' + last_date + ' Close:' + indicators_df['RSI'].iloc[-1] + ' Forecast('+str(look_back)+'):' + str(rsi_forecast_mean)

fig = make_subplots(rows=2, cols=1, row_heights=[0.6, 0.4], subplot_titles=(stock_title, indicators_title))


fig.add_trace(trace1,row=1, col=1)
fig.add_trace(trace2,row=1, col=1)
fig.add_trace(trace3,row=1, col=1)
fig.add_trace(trace4,row=1, col=1)

fig.add_trace(trace_train,row=2, col=1)
fig.add_trace(trace_test,row=2, col=1)
fig.add_trace(trace_prediction,row=2, col=1)
fig.add_trace(trace_forecast,row=2, col=1)

fig.add_trace(go.Scatter(
        x = [indicators_df_train.tail(len(indicators_df_test))['Date'].min(), np.max(indicators_forecast_dates)],
        y = [70, 70],
        mode = "lines",
        line = dict(shape = 'linear', color = 'rgb(100, 10, 100)', dash = 'dash')
    ),row=2, col=1)

fig.add_trace(go.Scatter(
        x = [indicators_df_train.tail(len(indicators_df_test))['Date'].min(), np.max(indicators_forecast_dates)],
        y = [50, 50],
        mode = "lines",
        line = dict(shape = 'linear', color = 'rgb(10, 12, 240)', dash = 'dash')
    ),row=2, col=1)

fig.add_trace(go.Scatter(
        x = [indicators_df_train.tail(len(indicators_df_test))['Date'].min(), np.max(indicators_forecast_dates)],
        y = [30, 30],
        mode = "lines",
        line = dict(shape = 'linear', color = 'rgb(10, 120, 24)', dash = 'dash')
    ),row=2, col=1)

fig.update_yaxes(showspikes=True, spikemode='across', spikesnap='cursor',spikedash='dash')
fig.update_xaxes(showspikes=True, spikemode='across', spikesnap='cursor', spikedash='dash')
fig.update_layout(xaxis_rangeslider_visible=False,height=650)

fig.show()
Path('/content/drive/MyDrive/models/charts/'+ last_date).mkdir(parents=True, exist_ok=True)
fig.write_html('/content/drive/MyDrive/models/charts/'+ last_date+ '/' + ticker + '_' + str(look_back) + '_08_inidicators_forecast.html')